In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
from sklearn import datasets

In [2]:
iris = datasets.load_iris()
data = iris.data
target = iris.target

In [41]:
params = np.random.randn(4,3)

In [3]:
# 소프트맥스 함수 
def softmax(data,params):
    # 데이터 포인트와 파라미터를 내적한다. 이 때, 파라미터 행렬은 ((변수의 갯수)행 * (분류 클래스)열)의 행렬이다
    # 예를들어, 변수가 4개이고, 클래스가 3개라면 파라미터 행렬은 4*3의 행렬이 된다.
    score = data.dot(params)
    # 각 클래스별 스코어값을 총합 스코어값으로 나누어준다. class = 1일때 해당 스코어벡터는 1열의 열벡터가 되는데
    # 이를 1열, 2열, 3열을 모두 더한 총합 스코어 열벡터로 나누어 표준화를 해주는 작업이다.(keepdims는 행과 열이 교환되지 않도록 한다)
    score = np.exp(score)/np.sum(np.exp(score),axis=1,keepdims=True)
    return(score)

In [4]:
# 경사하강법 함수

def gradient_descent(data,params,score,target,learning_rate,class_num):
    # 0으로만 차있는 껍데기온 배열을 만든다.
    gradient_vec = np.zeros((class_num,np.shape(data)[1]))
    # 변수 의존성을 피하기 위해 copy()함수로 복사한다.
    target_temp = target.copy()
    for i in range(0,class_num):
        # 자동 원핫 인코딩(1-1)
        # 분류 클래스만큼 순회를 돈다. 해당 클래스의 차례(i가 해당 클래스)가 아닌 경우 일괄적으로 0으로 처리해준다
        target_temp[target != i] = 0
        # i가 해당 클래스인 경우 1로 켜준다.
        target_temp[target == i] = 1
        # 그래디언트 벡터의 오차 부분을 정의해준다.(1)
        score[:,i] = score[:,i] - target_temp
    # 모든 분류 클래스의 순회가 끝나면, 앞서 구한 오차 부분과 데이터를 내적한다. (2)
    gradient_vec = data.T.dot(score) / np.shape(data)[0]
    # (1)과 (2)로 그래디언트 벡터를 정의하였다. 이제 학습률에 그래디언트 벡터를 곱하여 뉴턴법을 실행한다.
    params = params - learning_rate * gradient_vec
    return (params,gradient_vec)

In [23]:
#평가 함수(크로스 엔트로피)

def scoring(data,score,target,class_num):
    # 변수 의존성을 피하기 위해 copy()함수로 복사한다.
    target_temp = target.copy()
    # 변수 의존성을 피하기 위해 마찬가지로 copy()함수로 복사한다.
    score_temp = score.copy()
    for i in range(0,class_num):
        #자동 원핫 인코딩(1-1 참조)
        target_temp[target != i] = 0
        target_temp[target == i] = 1
        # 원핫 인코딩을 스위치처럼 활용한다. 로그 변환된 스코어 벡터는 만일 해당 데이터포인트가 순회하는 i에 해당하는 포인트가 아닐 경우
        # 0으로 꺼진다. 즉 [a,a,0,0,0,0,0],[0,0,b,b,b,0,0],[0,0,0,0,c,c] 꼴로 출력된다.
        score_temp[:,i] = np.log(score[:,i]) * target_temp
    # log 변환하며 nan값이 필연적으로 생성된다. nan값을 무시하기 위해 nansum으로 더해주고, 이를 총 샘플수로 나누어 표준화한 값을
    # 크로스 엔트로피 함수로 정의한다.
    cross_entrophy = -np.nansum(score_temp)/np.shape(data)[0]
    return(cross_entrophy)

In [103]:
def early_stopping(loss,minimum_loss,b):
    if loss > minimum_loss:
        b += 1
    if loss <= minimum_loss:
        minimum_loss = loss.copy()
        b = 0
    return (b,minimum_loss)

In [110]:
def softmax_regression(data,target,iteration,learning_rate):
    class_num = len(np.unique(target))
    # 최초 파라미터를 랜덤 선출한다.
    init_params = np.random.randn(np.shape(data)[1],len(np.unique(target)))
    params = init_params.copy()
    result = dict()
    minimum_loss = 100000
    b = 0
    for i in range(0,iteration):
        score = softmax(data,params)
        loss = scoring(data,score,target,class_num)
        print(i,loss)
        # 만약 손실함수가 10회 이상 최저값을 경신하지 못할경우 순회를 중지한다
        # 최저값을 경신할 경우 최저값을 갱신하고, b를 다시 0으로 초기화한다.
        b, minimum_loss = early_stopping(loss,minimum_loss,b)
        if b == 10:
            print("early stopping activated")
            break
        params,gradient_vec = gradient_descent(data,params,score,target,learning_rate,class_num)
    result["score"] = score
    result["params"] = params
    result["gradient_vec"] = gradient_vec
    result["init_params"] = init_params
    return(result)

In [111]:
result = softmax_regression(data,target,500,0.1)

0 1.3818764148669216
1 0.8613215322740342
2 0.8234378941558895
3 0.7927670876289302
4 0.7677569320401676
5 0.7487633159163992
6 0.7362925822747709
7 0.7332749405962883
8 0.7396242813790034
9 0.7564102497941549
10 0.7773484112444303
11 0.7888979457289715
12 0.8006243460213669
13 0.7851374188156641
14 0.7920795635967574
15 0.7641819934370244
16 0.7760683484014977
17 0.7436407940932338
early stopping activated


In [93]:
# 각 클래스 확률 출력

def class_proba(result,data):
    proba_vec = np.exp(np.dot(result["params"].T,data))/np.sum(np.exp(np.dot(result["params"].T,data)))
    return proba_vec

In [65]:
np.argmax(class_proba(result,data[139]))

2

In [76]:
class_proba(result,data[85]).round(3)

array([0.006, 0.941, 0.052])